Maintainer: Dennis Kibalama kibalama.3@osu.edu

This script is run with a configured anaconda/miniconda environment. To setup the same environment please run the commands below

`create --name sumo-env`

`conda activate sumo-env`

`conda install scipy numpy matplotlib termcolor jupyter ipympl`

In [2]:
# Import the required files and libraries

from __future__ import absolute_import
from __future__ import print_function
from termcolor import colored

import os, sys
import numpy as np
import scipy.io as scio

# check SUMO_HOME 
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
from sumolib import net, checkBinary

In [3]:

def geo_coordinates_extractor(file_name):
    ''' 
    A function that is extracts the GPS coordinates from a .mat file.

    Input arguments
        :<file_name>, a string of the source route file (excluding the .mat extension)
    
    Output arguments 
        :route_lat, a list of latitude geodetic coordinates
        :route_long, a list of longitude geodetic coordinates
    '''
    file_name = 'Route_19'

    parent_dir = os.getcwd()
    os.chdir("logs/gps/")
    gps_logs_dir = os.getcwd()
    route_lat = scio.loadmat(f"{file_name}.mat")['route_lat']
    route_long = scio.loadmat(f"{file_name}.mat")['route_long']

    os.chdir(parent_dir)

    return [
        route_long,
        route_lat
    ]

def create_net_object(sumo_map):

    '''
    Create a net object based on the name of the network
    Recall that the network is placed inside a folder with a similar name
    under the "env/traffic_model/" folder
    '''

    return net.readNet(f'env/traffic_model/{sumo_map}/{sumo_map}.net.xml')

In [4]:

# TODO
route_long, route_lat = geo_coordinates_extractor(file_name="Route_19") # <----- Enter the file containing GPS coordinates

# This command could take a while to run depending on the size of the SUMO network you load.
my_net = create_net_object(sumo_map="reduced_columbus_region") # <----- Assign the approprite name of the SUMO network file

In [4]:
def extract_edge_id(closest_edge):

    ''' 
    Convert the object contained in closest edge to a sting and extract the first item using the split() function
    The object uses xml syntax and is of the format

    <edge id="..." from="..." to="..."/>

    '''
    return str(closest_edge).split('"')[1]


def return_closest_edge(coordinates, my_net):

    ''' 
    This function takes the coordinates of the GPS points, and the network object and finds the closest edge to each point. 

    NOTE:This is an experimental implementation and still under development

    ----------
    known bugs
    ----------
    * the extract_edge_id() for some coordinates can return an empty list which implies that no matching edge will be found for the corresponding GPS coordinates.
    
    '''

    RADIUS = 0.1
    edge_list = []
    cnt = 0

    # For first index assign value to edge_list
    x, y = my_net.convertLonLat2XY(coordinates[0,0], coordinates[1,0])
    edges = my_net.getNeighboringEdges(x,y,RADIUS)

    # pick the closest edge
    if len(edges) > 0:
        distances_and_edges = sorted([(dist, edge) for edge, dist in edges])
        dist, closest_edge = distances_and_edges[0]

    edge_list.append(extract_edge_id(closest_edge))

    for item in range(1,coordinates.shape[1]):
        x, y = my_net.convertLonLat2XY(coordinates[0,item], coordinates[1,item])
        edges = my_net.getNeighboringEdges(x,y,RADIUS)

        # pick the closest edge
        if len(edges) > 0:
            distances_and_edges = sorted([(dist, edge) for edge, dist in edges])
            dist, closest_edge = distances_and_edges[0]

        # if item == 1: # Assign the id of the first closest edge to the edge_list 
        #     edge_list.append(extract_edge_id(closest_edge))
        #     # prev_edge_id = extract_edge_id(closest_edge)

        # else:
        #TODO check the returned closest edge against the previous edge and only append the list when the closest edge changes
        if edge_list[-1] != extract_edge_id(closest_edge):
            # Increment counter, change prev_edge_id for next iteration and append edge_list
            cnt += 1
            edge_list.append(extract_edge_id(closest_edge))
            # prev_edge_id = extract_edge_id(closest_edge)

    return edge_list

In [5]:
geo_coordinates = np.row_stack((route_long, route_lat))
route_edges = return_closest_edge(geo_coordinates, my_net)

In [ ]:
#TODO
# Write the extracted route edges to a .rou.xml file and run a simulation with SUMO to verify the route

In [19]:
input_string = str(route_edges[0])
result = input_string.split('"')[1]
result

'-21352789#22'